In [1]:
import os
os.chdir("../")

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
from chestCancerClassification.constants import *
from chestCancerClassification.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/abcdefya/Chest-Cancer-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub

In [9]:

dagshub.init(repo_owner='abcdefya', repo_name='Chest-Cancer-Classification', mlflow=True)

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        try:
            self.model = self.load_model(self.config.path_of_model)
            self._valid_generator()
            self.score = self.model.evaluate(self.valid_generator)
            self.save_score()
        except Exception as e:
            raise Exception(f"Error during evaluation: {str(e)}")

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        try:
            # Thiết lập tracking URI
            mlflow.set_tracking_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run(run_name="Chest_Cancer_Evaluation"):
                # Log các tham số từ config
                mlflow.log_params(self.config.all_params)
                
                # Log metrics
                mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

                # Log mô hình Keras
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model,
                        "model",
                        registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")

                # Log thêm artifacts (ví dụ: scores.json)
                mlflow.log_artifact("scores.json")
                
        except Exception as e:
            raise Exception(f"Error logging to MLflow: {str(e)}")



[2025-07-09 14:26:16,077: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as abcdefya

[2025-07-09 14:26:16,097: INFO: helpers: Accessing as abcdefya]
[2025-07-09 14:26:18,907: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/abcdefya/Chest-Cancer-Classification "HTTP/1.1 200 OK"]
[2025-07-09 14:26:21,794: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "abcdefya/Chest-Cancer-Classification"

[2025-07-09 14:26:21,805: INFO: helpers: Initialized MLflow to track repo "abcdefya/Chest-Cancer-Classification"]


Repository abcdefya/Chest-Cancer-Classification initialized!

[2025-07-09 14:26:21,809: INFO: helpers: Repository abcdefya/Chest-Cancer-Classification initialized!]


In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-07-09 14:26:28,217: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-09 14:26:28,219: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-09 14:26:28,222: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 33s 5s/step - loss: 23.2698 - accuracy: 0.4314
[2025-07-09 14:27:01,909: INFO: common: json file saved at: scores.json]


2025/07/09 14:27:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-07-09 14:27:08,096: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\dothe\AppData\Local\Temp\tmprwylq2em\model\data\model\assets
[2025-07-09 14:27:10,101: INFO: builder_impl: Assets written to: C:\Users\dothe\AppData\Local\Temp\tmprwylq2em\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/07/09 14:30:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
